In [5]:
import polars as pl
from src.paths import PYTHON_VULNERABILITY_FIXES_DATA_PATH
from github import Github, Auth
from os import environ
import gitlab
import jedi
import whatthepatch
from git import Repo
from pathlib import Path
from typing import Any
import jedi.api
import jedi.common
import os
import multiprocessing
import traceback

from src.process_code_changes import get_changes

## Filter out data

In [6]:
# python_commit_data = commit_data_only_top_langs.filter(pl.col("language") == "Python")
python_vulnerability_fixes = pl.read_parquet(PYTHON_VULNERABILITY_FIXES_DATA_PATH)
print(python_vulnerability_fixes.unique("vulnerability_id").shape[0])
python_vulnerability_fixes = python_vulnerability_fixes.group_by('vulnerability_id', 'repo', 'commit', 'pull_request_number', 'file', 'patch', 'patch_time', 'commit_source', 'file_extension', 'language').agg(pl.col("cwe_id"))
python_vulnerability_fixes = python_vulnerability_fixes.unique("patch")
print(python_vulnerability_fixes.unique("vulnerability_id").shape[0])


1805
1802


## Filter out data connected with test functionallity

In [7]:
python_vulnerability_fixes = python_vulnerability_fixes.filter(
    (
        pl.col("file").str.contains(r"\/{0,1}[tT][eE][sS][tT][sS]{0,1}\/")
        | pl.col("patch").str.contains("pytest")
        | pl.col("patch").str.contains("unittest")
    ).not_()
)
python_vulnerability_fixes.unique("vulnerability_id").shape[0]

1795

## Process missing commit

In [8]:
print(python_vulnerability_fixes.filter(pl.col("commit").is_null()))
python_vulnerability_fixes = python_vulnerability_fixes.with_columns(
    pl.when(pl.col("pull_request_number") == 24391)
    .then(pl.lit("86664c9405136a4904775c52e6caf100a474ec58"))
    .otherwise(pl.col("commit"))
    .alias("commit")
)
print(python_vulnerability_fixes.filter(pl.col("commit").is_null()))


shape: (1, 11)
┌────────────┬────────────┬────────┬────────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ vulnerabil ┆ repo       ┆ commit ┆ pull_reque ┆ … ┆ commit_so ┆ file_exte ┆ language ┆ cwe_id    │
│ ity_id     ┆ ---        ┆ ---    ┆ st_number  ┆   ┆ urce      ┆ nsion     ┆ ---      ┆ ---       │
│ ---        ┆ str        ┆ str    ┆ ---        ┆   ┆ ---       ┆ ---       ┆ str      ┆ list[str] │
│ str        ┆            ┆        ┆ i64        ┆   ┆ str       ┆ str       ┆          ┆           │
╞════════════╪════════════╪════════╪════════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ 2021-3733  ┆ python/cpy ┆ null   ┆ 24391      ┆ … ┆ github    ┆ py        ┆ Python   ┆ ["CWE-400 │
│            ┆ thon       ┆        ┆            ┆   ┆           ┆           ┆          ┆ "]        │
└────────────┴────────────┴────────┴────────────┴───┴───────────┴───────────┴──────────┴───────────┘
shape: (0, 11)
┌─────────────┬──────┬────────┬─────────────┬───┬────────────

In [9]:
auth = Auth.Token(environ["GITHUB_TOKEN"])
github_client = Github(auth=auth)
gl = gitlab.Gitlab(private_token=environ["GITLAB_TOKEN"])

In [10]:
# code_unit_changes: list[dict[str, Any]] = []
# code_context_changes: list[dict[str, Any]] = []


repos: dict[str, Repo] = {}

grouped_vulnerabilities = (
    python_vulnerability_fixes.group_by(
        "repo", "vulnerability_id", "commit", "commit_source"
    )
    .agg(pl.col("patch"), pl.col("file"), pl.col("language"))
    .sample(fraction=1, shuffle=True)
)
errors: list[dict[str, Any]] = []
# checked_commits = set([change["commit"] for change in code_unit_changes])


params: list[dict[str, Any]] = [
    (commit_data_row)
    for commit_data_row in grouped_vulnerabilities.iter_rows(named=True)
    # if commit_data_row["commit"] not in checked_commits
]


with multiprocessing.Pool(os.cpu_count()) as pool:
    for i in range(0, len(params), 20):
        results = pool.map(get_changes, params[i : i + 20 ])
        print(f"Processed {i} - {i+20} commits")
        print(results)
        for result in results:
            if isinstance(result, Exception):
                print(traceback.print_exception(result))

KeyboardInterrupt: 

In [1]:
checked_commits = set([change["commit"] for change in code_unit_changes])

NameError: name 'code_unit_changes' is not defined

In [30]:
print(repo_name)
print(fix_commit)

kovidgoyal/calibre
3a89718664cb8c


In [26]:
diffs = whatthepatch.parse_patch(patch)
list(diffs)[0].changes

[Change(old=177, new=177, line='        pass', hunk=1),
 Change(old=178, new=178, line='', hunk=1),
 Change(old=179, new=179, line='    def skip(self, type):', hunk=1),
 Change(old=180, new=None, line='        if type == TType.STOP:', hunk=1),
 Change(old=181, new=None, line='            return', hunk=1),
 Change(old=182, new=None, line='        elif type == TType.BOOL:', hunk=1),
 Change(old=None, new=180, line='        if type == TType.BOOL:', hunk=1),
 Change(old=183, new=181, line='            self.readBool()', hunk=1),
 Change(old=184, new=182, line='        elif type == TType.BYTE:', hunk=1),
 Change(old=185, new=183, line='            self.readByte()', hunk=1),
 Change(old=220, new=218, line='            for _ in range(size):', hunk=2),
 Change(old=221, new=219, line='                self.skip(etype)', hunk=2),
 Change(old=222, new=220, line='            self.readListEnd()', hunk=2),
 Change(old=None, new=221, line='        else:', hunk=2),
 Change(old=None, new=222, line='     

In [14]:
project = jedi.Project("repos/parisneo/lollms")
script = jedi.Script(
    path="repos/parisneo/lollms/lollms/server/endpoints/lollms_file_system.py", project=project)
name = script.goto(328, 23, follow_imports=True)[0]
name.type

'class'

In [304]:
script.get_context(27).parent().parent()

In [260]:
[name.module_name for name in script.goto(27, 32)]

['tornado.web']

In [214]:
[name for name in script.get_references(53, len("            callback = sanitize_")) if name.is_definition()]

[<Name full_name='code.autocomplete.server.sanitize_callback', description='def sanitize_callback'>]

In [37]:
commit_data_only_top_langs.unique(["cve_id", "commit"]).filter(pl.col("commit") == "63cde2daadc705edf086f2213b48c8c547f98358").head(50)

cve_id,repo,commit,language,reference,year,cwe_id,detection_date,score
str,str,str,str,str,i64,str,str,f64
"""2021-21695""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-59""","""2022-05-24T19:19:43.000000""",9.0
"""2021-21690""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-22""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21694""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-862""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21688""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-862""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21692""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-22""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21686""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-22""","""2022-05-24T19:19:45.000000""",9.0
"""2021-21693""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-285""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21691""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-59""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21689""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-862""","""2022-05-24T19:19:44.000000""",9.0


In [258]:
another_script = jedi.Script(path="repos/rtxteam/rtx/code/kg2c/create_indexes_constraints_canonicalized.py", project=project)
another_script.goto(102,36)

[]